In [1]:
%load_ext cython
%matplotlib inline
import sys
sys.path.append('/home/bram/ANTS')
sys.path.append('/home/bram/ANTS/entropy')
for p in sys.path:
    print(p)


/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages/IPython/extensions
/home/bram/.ipython
/home/bram/ANTS
/home/bram/ANTS/entropy


In [3]:
%%cython -a
# distutils: language = c++
from libcpp.vector cimport vector
from cythonic.plugins.db_controller cimport db_controller
from cythonic.plugins.db_path import db_path
from cythonic.plugins.queries import get_settings, get_steps
from cythonic.plugins.positions cimport point
from cythonic.core.domain cimport Domain

db = db_controller(db_path(), 'stigmergy.db')
def extract_settings(rows, headers):
    return {headers[i]:rows[0][i] for i in range(len(headers))}
cdef str qry = "SELECT * FROM sim"

# cdef point text2point(str txt):
#     cdef list lst = eval(txt)
#     cdef point result = {'x':lst[0], 'y':lst[1]}
#     return result

# data, headers = db.return_all(qry)
# print(extract_settings(headers, data))


cdef class SimPlayer:
    cdef:
        readonly object db
        readonly unsigned int id
        double evap_rate
        Domain domain
        
        double[:,::] positions
        double[:] headings
        double[:,::] lefts
        double[:,::] rights
        
        readonly list steps
    
    def __init__(self,sim_id, db_path, db_name):
        self.db = db_controller(db_path, db_name)
        self.id = sim_id
        d,h = db.return_all(get_settings(sim_id, 'sim_settings'))
        self.evap_rate = extract_settings(d,h)['evap_rate']
        sim_settings = extract_settings(*db.return_all(get_settings(sim_id,'sim_settings')))
        ant_settings = extract_settings(*db.return_all(get_settings(sim_id,'ant_settings')))
        dom_settings = extract_settings(*db.return_all(get_settings(sim_id,'domain_settings')))
        gauss_settings = extract_settings(*db.return_all(get_settings(sim_id,'gauss_settings')))

        " initialize the objects "
        self.init_domain(dom_settings)
        self.init_gaussian(gauss_settings)
        
        
#     def init_sim(self, dict sim_dict):
#         " extract the simulation settings "
#         self.
        
    def init_domain(self,dict domain_dict):
        " extract the query into useable initialization parameters "
        del domain_dict['sim_id'] # remove sim_id from dict
        cdef dict dom_dict = {}
        for key, value in domain_dict.items():
            if type(value) is str:
                dom_dict[key] = eval(value)
            else:
                dom_dict[key] = value
        self.domain = Domain(**dom_dict)
        
    def init_gaussian(self, gauss_dict):
        del gauss_dict['sim_id'] # remove sim_id from dict
        self.domain.init_gaussian(gauss_dict['covariance'], gauss_dict['significancy'])
        
    
    def get_steps(self ):
        self.steps, _ = db.return_all(get_steps(self.id))
        
        #placeholders
#         self.step_nr = np.zeros(self.)
#         self.pos_list
        
        
        
from pandas import read_sql, DataFrame as DF
print(get_settings('5',table='sim_settings'))
player = SimPlayer(70,db_path(), 'stigmergy.db')
df =read_sql(get_steps(player.id), player.db.db, )
print(df)

SELECT * FROM sim_settings WHERE sim_settings.sim_id = 5
       STEP_NR  ANT_ID           X           Y       THETA  Q
0            0       0  282.161716  458.505390  309.082429  1
1            1       0  283.783687  456.602968  310.450313  1
2            2       0  285.450674  454.739864  311.820166  1
3            3       0  287.161724  452.917144  313.190020  1
4            3       1  276.525163  545.297644   55.310731  1
5            3       2  197.804664  505.014901  187.223685  1
6            3       3  224.640520  454.122622  226.864359  1
7            4       0  288.915823  451.135814  314.558694  1
8            4       1  278.106650  547.233851   50.758171  1
9            4       2  195.463984  504.136711  200.565381  1
10           4       3  222.519450  452.799345  211.958899  1
11           4       4  258.970649  448.303810  288.696779  1
12           5       0  290.712018  449.396940  315.928975  1
13           5       1  279.836917  549.038339   46.202898  1
14           

In [ ]:
%%cython -a
from libc.math cimport log2 as clog2
import numpy as np
cdef double[:,::] M = np.random.rand(20,20)
cdef unsigned int i,j
(i,j) = M.shape[:2]

cdef double csum(double[:,::] M):
    cdef unsigned int i,j, I,J
    (I,J) = M.shape[:2]
    cdef double s=0.
    for i in range(I):
        for j in range(J):
            s+=M[i,j]
    return s

cdef double H(double[:,::] M):
    cdef unsigned int i,j,I,J
    (I,J) = M.shape[:2]
    cdef double h = 0.
    cdef double T = csum(M)
    cdef str err_txt = 'log of nearnegative number is ill-defined'
    if T <= 0.0:
        raise ValueError('Sum of pheromone map <= 0')
    for i in range(I):
        for j in range(J):
            if M[i,j]<= 1e-10:
                raise ValueError(err_txt)
            h-= M[i,j]/T*clog2(M[i,j]/T)
    return h

cdef double ent = H(M)
print(ent)

cdef unsigned int[:] rw = np.arange(0,10,1,dtype=np.uint32)
print(np.asarray(rw).tolist())